In [14]:
import json
from environments.rush_hour_env import RushHourEnv
from agents.random_agent import RandomAgent

# 데이터 로드
with open("data/vehicle_data.json", "r") as file:
    vehicle_data = json.load(file)

# 환경 초기화
env = RushHourEnv(vehicle_data=vehicle_data)
agent = RandomAgent(env.action_space)

state = env.reset()
done = False

while not done:
    action = agent.choose_action()
    next_state, reward, done, _ = env.step(action)
    print(f"Action: {action}, Reward: {reward}")
    env.render()

env.close()


Action: [8 1], Reward: -1
Action: [5 1], Reward: -1
Action: [0 1], Reward: -1
Action: [11  0], Reward: -1
Action: [7 0], Reward: -1


IndexError: index 8 is out of bounds for axis 0 with size 8